In [1]:
import cv2
import numpy as np
import os
is_out_exist = os.path.exists('out')
if not is_out_exist:
    # Create a new directory because it does not exist
    os.makedirs('out')
    print("The new directory is created!")

In [2]:
def MSE(img1, img2):
    #return np.sum((img1-img2)**2)/img1.shape[0]/img1.shape[1]/img1.shape[2]
    return np.mean((img1 - img2) ** 2)    
def show_image(img, name):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def test_color(pixel,colors):
        inside_colors = False
        for color in colors:
            if color[0] == pixel[0] and color[1] == pixel[1] and color[2] == pixel[2]:
                inside_colors = True
        if inside_colors == False:
            raise Exception('fuck up')

In [3]:
def re_construct_img(new_img, original_img):
    res = np.zeros(original_img.shape, dtype=np.uint8)
    colors = []
    for bin in new_img:
        color = bin[0]
        #print(color[0:3])
        colors.append(color[0:3])
        for pixel in bin:
            if color[0]!=pixel[0] or color[1]!=pixel[1] or color[2]!=pixel[2]:
                raise Exception("fuck up")
            res[pixel[3], pixel[4]] = color[0:3]
    return res, colors
def generate_indexed_img(img):
    index_i_j = 2
    img_with_index = np.zeros([img.shape[0],img.shape[1], img.shape[2]+index_i_j], dtype=int)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            u = np.zeros([img.shape[2]+index_i_j], dtype=int)
            u[0] = img[i,j,0]
            u[1] = img[i,j,1]
            u[2] = img[i,j,2]
            u[3] = i
            u[4] = j
            img_with_index[i,j] = u
    return img_with_index.reshape(-1, img_with_index.shape[2])
def median_cut_quantization(img,current_level, n, bins):
    #print('here',img.shape)
    if current_level == n:
        #find the mean of the img
        rmean = np.mean(img[:,0])
        gmean = np.mean(img[:,1])
        bmean = np.mean(img[:,2])
        #print('in max level ',img.shape, ' quantized color: ',rmean,gmean,bmean)
        #print(img)
        img[:,0] = rmean
        img[:,1] = gmean
        img[:,2] = bmean
        #print(img)
        
        bins.append(img)
        return
    #find the longest side of the img
    def find_widest_side(img):
        r_gap = np.max(img[:,0]) - np.min(img[:,0])
        g_gap = np.max(img[:,1]) - np.min(img[:,1])
        b_gap = np.max(img[:,2]) - np.min(img[:,2])
        if r_gap >=g_gap and r_gap>=b_gap:
            return 0
        elif g_gap >=r_gap and g_gap>=b_gap:
            return 1
        else:
            return 2
    widest_side_color = find_widest_side(img)
    
    #find the median
    widest_side = img[:,widest_side_color]
    sorted_widest_side = np.sort(widest_side)
    median = sorted_widest_side[int(sorted_widest_side.shape[0]/2)]
    
    #split the img into two parts
    cub_a = img[img[:,widest_side_color]<=median]
    cub_b = img[img[:,widest_side_color]>median]
    #recursive call the function
    median_cut_quantization(cub_a,current_level+1,n,bins)
    median_cut_quantization(cub_b,current_level+1,n,bins)
def ordered_dither(img, dither_matrix):
    res = np.zeros(img.shape, dtype=int)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for k in range(img.shape[2]):
                if img[i,j,k] > dither_matrix[i%dither_matrix.shape[0],j%dither_matrix.shape[1]]:
                    res[i,j,k] = 255
                else:
                    res[i,j,k] = 0
    return res
def diffusion_dither(img, colors):
    def pick_closest_color(pixel,colors):
        min_distance = 255*255*3
        if len(colors) == 0:
            raise Exception("no color")
        for color in colors:
            color_distance = np.sum((pixel-color)**2)
            if color_distance < min_distance:
                min_distance = color_distance
                closest_color = color
        return closest_color

    res = img.copy().astype(float)
    
    for i in range(res.shape[0]):
        for j in range(res.shape[1]):
            target_color = pick_closest_color(res[i,j],colors)
            test_color(target_color,colors)
            for channel in range(3):#rgb
                error = img[i,j,channel] - target_color[channel]
                if j+1 < res.shape[1]:
                    res[i,j+1,channel] = res[i,j+1,channel] + error*7/16
                if i+1 < res.shape[0]:
                    res[i+1,j,channel] = res[i+1,j,channel] + error*5/16
                if i+1 < res.shape[0] and j-1 >= 0:
                    res[i+1,j-1,channel] = res[i+1,j-1,channel] + error*3/16
                if i+1 < res.shape[0] and j+1 < res.shape[1]:
                    res[i+1,j+1,channel] = res[i+1,j+1,channel] + error*1/16
            res[i,j] = target_color
            test_color(res[i,j],colors)
    return res.astype(np.uint8)

def test_color(pixel,colors):
        inside_colors = False
        for color in colors:
            if color[0] == pixel[0] and color[1] == pixel[1] and color[2] == pixel[2]:
                inside_colors = True
        if inside_colors == False:
            raise Exception('fuck up')

In [4]:
target_image = '.\\img\\Lenna.jpg'
img = cv2.imread(target_image, cv2.IMREAD_COLOR)
indexed_image = generate_indexed_img(img)

## define `n`, which sets the color box's size

In [11]:
n = 3 #change this to change the number of colors

In [12]:
bins = []
median_cut_quantization(indexed_image,0,n,bins)
median_cut_quantization_img, colors = re_construct_img(bins,img)
print('mse ' +'in n = '+str(n)+' after the image are quantized by median cut: {}'.format(MSE(img, median_cut_quantization_img)))

mse in n = 3 after the image are quantized by median cut: 66.89740559739359


In [13]:
diffusion_dither_img = diffusion_dither(img, colors)
#show_image(diffusion_dither_img, 'error diffusion dithering')
img = cv2.imread(target_image, cv2.IMREAD_COLOR)
print('mse ' +'in n = '+str(n)+' after the image are quantized by median cut with the help of diffusion dithering: {}'.format(MSE(img, diffusion_dither_img)))

mse in n = 3 after the image are quantized by median cut with the help of diffusion dithering: 67.38919711050579


In [14]:
cv2.imwrite('.\\out\\median_cut'+ str(n) +'.png', median_cut_quantization_img)
cv2.imwrite('.\\out\\error_diffusion_dithering_' + str(n) +'.png', diffusion_dither_img)

True

## verify the saved image and output mse with `test_color`

In [15]:
for color in colors:
    print(color)

[62 24 99]
[ 89  48 116]
[ 78  67 175]
[ 95  88 191]
[119 117 193]
[107 117 222]
[127 151 225]
[169 187 226]


In [16]:
original_img = cv2.imread('.\\img\\Lenna.jpg', cv2.IMREAD_COLOR)
for name in ['median_cut'+ str(n) +'.png', 'error_diffusion_dithering_'+str(n)+'.png']:
    output_dir = '.\\out\\'
    img = cv2.imread(output_dir+name, cv2.IMREAD_COLOR)
    #print(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            test_color(img[i,j],colors)
    print('mse ' + name +' : {}'.format(MSE(img, original_img)))
    #show_image(img, name)

mse median_cut3.png : 66.94910671366767
mse error_diffusion_dithering_3.png : 67.38919711050579
